In [1]:
import pandas as pd

df = pd.read_csv('/content/main_task.xls',sep=",")
# Какие столбцы НЕ содержат пропущенных (None) значений?
df.dropna(axis=1)

,Restaurant_id,City,Ranking,Rating,Reviews,URL_TA,ID_TA
0,id_5569,Paris,5570.0,3.5,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,1537.0,4.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,353.0,4.5,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,3458.0,5.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,621.0,4.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963
...,...,...,...,...,...,...,...
39995,id_499,Milan,500.0,4.5,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414
39996,id_6340,Paris,6341.0,3.5,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036
39997,id_1649,Stockholm,1652.0,4.5,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615
39998,id_640,Warsaw,641.0,4.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838


In [2]:
# В каких столбцах данные хранятся в числовом формате?
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [3]:
# В каких столбцах хранящиеся данные представляют собой список?
# ни в одном

In [4]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [5]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [6]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [7]:
# Создаём, обучаем и тестируем модель
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [8]:
# Чтобы код, отвечающий за обучение модели, заработал, мы должны использовать в нём датафрейм, содержащий только количественные признаки и не содержащий None-значений
# Удалим столбцы, содержащие данные типа object
X_test = X_test.drop(['City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA'], axis=1)
X_test

,Ranking,Number of Reviews
8575,5195.0,7.0
11640,769.0,51.0
22314,5844.0,35.0
33302,3218.0,18.0
5429,2796.0,26.0
...,...,...
1703,4720.0,7.0
6725,731.0,22.0
11606,7605.0,68.0
39437,1276.0,180.0


In [9]:
# заполним пропущенные значения (None или NaN) средним арифметическим значением или нуль для всего столбца
X_test = X_test.rename({'Ranking': 'Ranking', 'Number of Reviews': 'Number_of_Reviews'}, axis=1)
X_test = pd.DataFrame(X_test)
X_test

,Ranking,Number_of_Reviews
8575,5195.0,7.0
11640,769.0,51.0
22314,5844.0,35.0
33302,3218.0,18.0
5429,2796.0,26.0
...,...,...
1703,4720.0,7.0
6725,731.0,22.0
11606,7605.0,68.0
39437,1276.0,180.0


In [10]:
X_test.Ranking = X_test.Ranking.fillna(sum(X_test.Ranking)/len(X_test.Ranking))
X_test.Number_of_Reviews = X_test.Number_of_Reviews.fillna('0')
X_test

,Ranking,Number_of_Reviews
8575,5195.0,7
11640,769.0,51
22314,5844.0,35
33302,3218.0,18
5429,2796.0,26
...,...,...
1703,4720.0,7
6725,731.0,22
11606,7605.0,68
39437,1276.0,180


In [11]:
# То же самое для X_train
X_train= X_train.drop(['City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA'], axis=1)
X_train = X_train.rename({'Ranking': 'Ranking', 'Number of Reviews': 'Number_of_Reviews'}, axis=1)
X_train = pd.DataFrame(X_train)
X_train.Ranking = X_train.Ranking.fillna(sum(X_train.Ranking)/len(X_train.Ranking))
X_train

,Ranking,Number_of_Reviews
32043,490.0,153.0
20026,4530.0,108.0
11773,2812.0,198.0
24181,481.0,204.0
24737,685.0,58.0
...,...,...
18640,126.0,618.0
24961,4814.0,72.0
33741,394.0,347.0
27638,9262.0,12.0


In [12]:
# заполним пропущенные значения (None или NaN) нулем для всего столбца
X_train.Number_of_Reviews = X_train.Number_of_Reviews.fillna('0')

In [13]:
# заполним пропущенные значения (None или NaN) каким-то одним значением средним арифметическим для всего столбца
y_train = y_train.fillna(sum(y_train)/len(y_train))
y_train

32043    4.0
20026    4.5
11773    4.0
24181    4.5
24737    4.5
        ... 
18640    4.0
24961    4.5
33741    4.0
27638    4.0
19221    5.0
Name: Rating, Length: 30000, dtype: float64

In [14]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [15]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.427887545959596


In [16]:
# Повысить точность прогнозов
# Сколько вариантов непустых значений встречается в столбце Price Range?
df['Price Range'].value_counts(dropna = False)

$$ - $$$    18412
NaN         13886
$            6279
$$$$         1423
Name: Price Range, dtype: int64

In [17]:
df.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


In [18]:
# Сколько городов представлено в наборе данных?
len(df['City'].value_counts())

31

In [19]:
# Сколько типов кухонь представлено в наборе данных?
import re

df['Cuisine Style'] = df['Cuisine Style'].fillna(value='other')
# Посчитаем сколько всего видов кухонь и составим из них список
pattern = re.compile('\w+\s*\w+\s*\w*')
cuisines =[]
for string in df['Cuisine Style']:
  if string != 'other':
    c = pattern.findall(string)
  cuisines.append(c)
# Создадим список с уникальными значениями
cuis = set()
for item in cuisines:
  for i in item:
    cuis.add(i)
cuis.add(i)
# общее количество уникальных кухонь
len(cuis)

126

In [20]:
# Какая кухня представлена в наибольшем количестве ресторанов? 
cuisin ={}
for i in cuis: 
  cuisin[i] = 0
for item in cuisines:
  for i in item:
    cuisin[i] += 1
cuisin

{'Afghani': 25,
 'African': 194,
 'Albanian': 13,
 'American': 1687,
 'Arabic': 63,
 'Argentinean': 212,
 'Armenian': 21,
 'Asian': 3925,
 'Australian': 32,
 'Austrian': 509,
 'Azerbaijani': 4,
 'Balti': 115,
 'Bangladeshi': 97,
 'Bar': 4306,
 'Barbecue': 716,
 'Belgian': 350,
 'Brazilian': 158,
 'Brew Pub': 287,
 'British': 2091,
 'Burmese': 2,
 'Cafe': 3048,
 'Cajun ': 26,
 'Cambodian': 33,
 'Canadian': 8,
 'Caribbean': 125,
 'Caucasian': 5,
 'Central American': 171,
 'Central Asian': 17,
 'Central European': 1797,
 'Chilean': 10,
 'Chinese': 1492,
 'Colombian': 22,
 'Contemporary': 706,
 'Creole': 26,
 'Croatian': 38,
 'Cuban': 33,
 'Czech': 761,
 'Danish': 232,
 'Delicatessen': 532,
 'Diner': 378,
 'Dutch': 397,
 'Eastern European': 629,
 'Ecuadorean': 6,
 'Egyptian': 28,
 'Ethiopian': 56,
 'European': 13146,
 'Fast Food': 2260,
 'Filipino': 13,
 'French': 4177,
 'Fujian': 3,
 'Fusion': 745,
 'Gastropub': 619,
 'Georgian': 24,
 'German': 858,
 'Gluten Free Options': 5364,
 'Greek':

In [21]:
# Какое среднее количество кухонь предлагается в одном ресторане? Если в данных отсутствует информация о типах кухонь, то считайте, что в этом ресторане предлагается только один тип кухни. Ответ округлите до одного знака после запятой
cu = []
cui=[]
for string in df['Cuisine Style']:
  c = pattern.findall(string)
  cu.append(c) # список из всех видов кухонь включая other
for item in cu:
  for i in item:
    cui.append(i)
cui.append(i)# список со всеми кухнями (не уникальный)
# общее количество кухонь делить на количество ресторанов
len(cui)/40000


2.622925

In [22]:
pattern2 = re.compile('\d{2}/\d{2}/\d{4}')
from datetime import datetime, timedelta
#Записываем даты в список
dates = []
for string in df['Reviews']:
    d = pattern2.findall(string)
    dates.append(d)
#Переводим даты в формат datetime
datetime_list = []
for line in dates:
    if len(line)>=2:
        new_date_list = []
        for date in line:
            date = date[0:10]
            dt = datetime.strptime(date, '%m/%d/%Y')
            new_date_list.append(dt)
        datetime_list.append(new_date_list)
#Находим разницу в днях между датами каждого второго отзыва с предыдущим
delta_lst = []
for i in range(0,len(datetime_list)):
    delta = datetime_list[i][0] - datetime_list[i][1]
    delta_lst.append(delta)

# Какое максимальное количество дней отделяет даты публикации отзывов, размещённых на сайте ресторана? 
abs(max(delta_lst))
# Когда был оставлен самый свежий отзыв? 
max(datetime_list)

[datetime.datetime(2018, 2, 26, 0, 0), datetime.datetime(2018, 2, 26, 0, 0)]

In [23]:
# Преобразуем столбец Price Range. Пусть у нас будет три категории, но числовые
df['price']= df['Price Range']
df.price.loc[df['Price Range']=='$'] = 1
df.price.loc[df['Price Range']=='$$ - $$$'] = 2
df.price.loc[df['Price Range']=='$$$$'] = 4
df['price']=pd.to_numeric(df['price'])
df['price']

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


0        2.0
1        NaN
2        4.0
3        NaN
4        2.0
        ... 
39995    2.0
39996    2.0
39997    NaN
39998    2.0
39999    2.0
Name: price, Length: 40000, dtype: float64

In [24]:
# Представлен 31 город, давайте зашируем их от 1 до 31
df['code_city']= df['City']
df.code_city.loc[df['City']=='London'] = 1
df.code_city.loc[df['City']=='Paris'] = 2
df.code_city.loc[df['City']=='Madrid'] = 3
df.code_city.loc[df['City']=='Barcelona'] = 4
df.code_city.loc[df['City']=='Berlin'] = 5
df.code_city.loc[df['City']=='Milan'] = 6
df.code_city.loc[df['City']=='Rome'] = 7
df.code_city.loc[df['City']=='Prague'] = 8
df.code_city.loc[df['City']=='Lisbon'] = 9
df.code_city.loc[df['City']=='Vienna'] = 10
df.code_city.loc[df['City']=='Amsterdam'] = 11
df.code_city.loc[df['City']=='Brussels'] = 12
df.code_city.loc[df['City']=='Hamburg'] = 13
df.code_city.loc[df['City']=='Munich'] = 14
df.code_city.loc[df['City']=='Lyon'] = 15
df.code_city.loc[df['City']=='Stockholm'] = 16
df.code_city.loc[df['City']=='Budapest'] = 17
df.code_city.loc[df['City']=='Warsaw'] = 18
df.code_city.loc[df['City']=='Dublin'] = 19
df.code_city.loc[df['City']=='Copenhagen'] = 20
df.code_city.loc[df['City']=='Athens'] = 21
df.code_city.loc[df['City']=='Edinburgh'] = 22
df.code_city.loc[df['City']=='Zurich'] = 23
df.code_city.loc[df['City']=='Oporto'] = 24
df.code_city.loc[df['City']=='Geneva'] = 25
df.code_city.loc[df['City']=='Krakow'] = 26
df.code_city.loc[df['City']=='Oslo'] = 27
df.code_city.loc[df['City']=='Helsinki'] = 28
df.code_city.loc[df['City']=='Bratislava'] = 29
df.code_city.loc[df['City']=='Luxembourg'] = 30
df.code_city.loc[df['City']=='Ljubljana'] = 31
df['code_city']=pd.to_numeric(df['code_city'])
df['code_city']

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


0         2
1        16
2         1
3         5
4        14
         ..
39995     6
39996     2
39997    16
39998    18
39999     6
Name: code_city, Length: 40000, dtype: int64

In [25]:
# Добавим столбец с количеством кухонь в ресторане
len_cu_1=[]
for i in range(0, len(df['Cuisine Style'])):
  len_cu=len(cu[i])
  len_cu_1.append(len_cu)
df['len_Cuisine'] = len_cu_1


In [26]:
# Пробуем еще раз вычислить МАЕ
import numpy as np
from scipy import stats
X_1 = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y_1 = df['Rating']
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=0.25)
X_train_1 = X_train_1.drop(['City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA'], axis=1)
X_test_1 = X_test_1.drop(['City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA'], axis=1)
# заменим na в price на моду
X_train_1.price = X_train_1.price.fillna(int(stats.mode(X_train_1.price)[0]))

In [43]:
# заменим na в Number of Reviews на 0
X_train_1 = X_train_1.rename({'Number of Reviews': 'Number_of_Reviews'}, axis=1)
X_train_1.Number_of_Reviews = X_train_1.Number_of_Reviews.fillna(0)


In [51]:
X_test_1 = X_test_1.rename({'Number of Reviews': 'Number_of_Reviews'}, axis=1)
X_test_1.Number_of_Reviews = X_test_1.Number_of_Reviews.fillna(0)
X_test_1.price = X_test_1.price.fillna(int(stats.mode(X_test_1.price)[0]))

In [60]:
# Создаём модель
regr_1 = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr_1.fit(X_train_1, y_train_1)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred_1 = regr_1.predict(X_test_1)

In [61]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test_1, y_pred_1))

MAE: 0.21502299999999996
